<a href="https://www.kaggle.com/code/puneetkoundal/fine-tune-gemma2b-model-using-keras-using-lora?scriptVersionId=217951826" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 1.INTRODUCTION

Gemma2b is a state-of-the-art transformer-based model designed to handle a wide range of natural language processing (NLP) tasks. It belongs to the family of large language models that leverage vast amounts of training data and computational power to achieve remarkable performance across tasks like:

Text Classification: Assigning categories or labels to text (e.g., sentiment analysis, topic detection).
Text Generation: Generating coherent and contextually appropriate text (e.g., conversational agents, content creation).
Summarization: Producing concise summaries from long documents or articles.
Translation: Translating text between languages.
Question Answering: Answering questions based on a given context or dataset.
The model has been pretrained on massive text datasets, capturing a deep understanding of language structure, grammar, semantics, and even domain-specific knowledge.

# 2.Prerequisites

1. Sign in to Kaggle
Visit Kaggle.
Log in to your Kaggle account.
2. Access API Credentials
Click on your profile picture in the top-right corner.
From the dropdown menu, select "Account".
Scroll down to the "API" section.
Click the "Create New API Token" button.
3. Download kaggle.json
This will download a file named kaggle.json containing your Kaggle username and Kaggle API key in JSON format.
4. Use Credentials in Code
You can directly upload kaggle.json to Colab and set environment variables:

In [1]:

import os
import json

# Load kaggle.json
with open('/kaggle/input/kaggle-json/kaggle (1).json', 'r') as f:
    kaggle_data = json.load(f)

# Set environment variables
os.environ['KAGGLE_USERNAME'] = kaggle_data['username']
os.environ['KAGGLE_KEY'] = kaggle_data['key']

#print("Kaggle credentials successfully set!")      #UNCOM


In [2]:
import os
import json

# Load kaggle.json
with open('/kaggle/input/kaggle-json/kaggle (1).json', 'r') as f:
    kaggle_data = json.load(f)

# Set environment variables
os.environ['KAGGLE_USERNAME'] = kaggle_data['username']
os.environ['KAGGLE_KEY'] = kaggle_data['key']

#print("Kaggle credentials successfully set!")      #UNCOM


# 3.Setting Up the Environment

Libraries Overview:
os: Provides functions to interact with the operating system, such as file handling and environment variable management.
enum: Allows for the creation of enumerations, which are a set of symbolic names bound to unique values.
re: Provides regular expression matching operations for pattern matching in strings.
string: Contains string constants and classes for string manipulation.
chex: A utility library for JAX-related development that helps with debugging, type checking, and testing.
jax: A library for high-performance numerical computing and machine learning using automatic differentiation and GPU/TPU acceleration.
jax.numpy: Provides a NumPy-like interface for tensor operations using JAX.
optax: A gradient processing and optimization library for JAX, commonly used for defining and applying optimization algorithms.
tensorflow: A framework for building and deploying machine learning models.
tensorflow_datasets: A library providing easy access to a wide variety of datasets for machine learning experiments.

In [3]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U "keras>=3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.5 MB/s eta 0:00:00


In [4]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [5]:
import keras
import keras_nlp

# 4.Load the Gemma model

In [6]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

# 5. Data prepration

 This notebook is a work-in-progress and not yet complete.

 
 Certain sections may be unfinished, and results may not be final.

 
 I will continue to update and refine the notebook.